In [2]:
!pip install tensorflow===2.12.0
!pip install torch
!pip install keras===2.10.0
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install nltk
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.10.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [3]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

## 02.03. Summarization with Pipelines

In [4]:
verbose_text ="""
Earth is the third planet from the Sun and the only astronomical object
known to harbor life.
While large volumes of water can be found
throughout the Solar System, only Earth sustains liquid surface water.
About 71% of Earth's surface is made up of the ocean, dwarfing
Earth's polar ice, lakes, and rivers.
The remaining 29% of Earth's
surface is land, consisting of continents and islands.
Earth's surface layer is formed of several slowly moving tectonic plates,
interacting to produce mountain ranges, volcanoes, and earthquakes.
Earth's liquid outer core generates the magnetic field that shapes Earth's
magnetosphere, deflecting destructive solar winds.
"""

verbose_text = verbose_text.replace("\n","")


In [5]:
from transformers import pipeline


extractive_summarizer = pipeline("summarization",
                                 min_length=10,
                                 max_length=100)

#Extractive summarization
extractive_summary=extractive_summarizer(verbose_text)

print(extractive_summary[0].get("summary_text"))



 Earth is the third planet from the Sun and the only astronomical object known to harbor life . About 71% of Earth's surface is made up of the ocean, dwarfing Earth's polar ice, lakes, and rivers . The remaining 29% of the surface is land, consisting of continents and islands .


In [6]:
print("Checkpoint used: ", extractive_summarizer.model.config)

Checkpoint used:  BartConfig {
  "_name_or_path": "sshleifer/distilbart-cnn-12-6",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_en

## 02.05 Evaluating with ROUGE

In [7]:
import evaluate

rouge_evaluator = evaluate.load("rouge")

#Evaluate exact match strings
reference_text=["This is the same string"]
predict_text=["This is the same string"]

eval_results=rouge_evaluator.compute(predictions=predict_text,
                                     references=reference_text)
print("Results for Exact match",eval_results)


Results for Exact match {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


In [8]:
#Evaluate no-match strings
reference_text=["This is the different string"]
predict_text=["Google can predict warm weather"]

eval_results=rouge_evaluator.compute(predictions=predict_text,
                                     references=reference_text)
print("\nResults for no match", eval_results)


Results for no match {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [9]:

#Evaluate summary
eval_results=rouge_evaluator.compute(
    predictions=[extractive_summary[0].get("summary_text")],
    references=[verbose_text])

print("\nResults for Summary generated", eval_results)


Results for Summary generated {'rouge1': 0.6369426751592356, 'rouge2': 0.5935483870967743, 'rougeL': 0.624203821656051, 'rougeLsum': 0.624203821656051}
